# Setting Pyspark in Colab

## Spark 설치 및 초기 설정
 Spark은 대용량 데이터 처리를 위한 분산 컴퓨팅 프레임워크입니다


In [ ]:
#!sudo apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.2.4/spark-3.2.4-bin-hadoop3.2.tgz
!tar xf spark-3.2.4-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
# findspark를 사용해 Spark 초기화
import findspark
findspark.init("/content/spark-3.2.4-bin-hadoop3.2")

In [ ]:
# SparkSession과 SparkContext를 설정합니다.
from pyspark.sql import SparkSession

# SparkSession 생성
spark = SparkSession.builder.appName("Friends Data Analysis").getOrCreate()


## 데이터 불러오기 및 SQL 연습

### TODO: 참고 주석과 pandas에 대한 지식을 기반으로 빈칸을 채워주세요.

In [ ]:
# 데이터 스키마 설정
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
# StructType과 StructField는 데이터 스키마를 정의할 때 사용됩니다.
# 참조: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.types.StructType

# SparkSession 생성
spark = SparkSession.builder.getOrCreate()
# 기존 세션을 사용하거나 새로운 세션을 생성합니다.

# 데이터 읽기 옵션 설정
option1 = "header"; argument1 = "False"  # CSV 파일에 헤더가 포함되지 않았음을 명시
option2 = "inferSchema"; argument2 = "true"  # 데이터 타입을 자동으로 추론하도록 설정
option3 = "delimiter"; argument3 = ","  # 쉼표를 구분자로 지정

# 스키마 정의
fields = [
    ___________("index", IntegerType(), True),  # 인덱스 열 정의
    ___________("name", StringType(), True),  # 이름 열 정의
    ___________("age", IntegerType(), True),  # 나이 열 정의
    ___________("numOfFriends", IntegerType(), True)  # 친구 수 열 정의
]
schema = StructType(fields)  # 정의된 필드로 스키마 생성

# CSV 파일 읽기
df = spark.read.schema(schema).format("csv")\
    .option(option1, argument1)\
    .option(option2, argument2)\
    .option(option3, argument3)\
    .load("fakefriends.csv")
# read.schema()는 스키마를 설정하여 CSV 파일을 읽는 메서드입니다.
# 참조: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader

# 특정 열 선택 및 데이터 보기
# 참조: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.select
df.select("name", "age").show()  # name과 age 열만 선택하여 표시
# select()는 특정 열을 선택하여 반환하는 메서드입니다.



In [ ]:
# show()는 DataFrame을 콘솔에 표시합니다.
# 참조: https://downloads.apache.org/spark/docs/3.2.4/api/python/reference/api/pyspark.sql.DataFrame.html
df.show()  # 데이터 미리보기
df.take(10)  # 상위 10개 행 가져오기
df.printSchema()  # 스키마 출력
df.describe().show()  # 요약 통계


In [ ]:
# 정렬 및 중복 제거
# sort()는 지정된 열을 기준으로 DataFrame을 정렬합니다.
# 참조: https://downloads.apache.org/spark/docs/3.2.4/api/python/reference/api/pyspark.sql.DataFrame.sort
df.___________("name").show()  # 이름 기준으로 정렬

# dropDuplicates()는 중복 행을 제거합니다.
unique_df = df.___________()  # 중복 제거

unique_df.___________("name").show()  # 중복 제거된 데이터 정렬


In [ ]:
# 그룹화 및 집계
import pyspark.sql.functions as sf

# groupby()는 지정된 열을 기준으로 데이터를 그룹화합니다.
# agg()는 집계 함수를 적용합니다.
# 참조: https://downloads.apache.org/spark/docs/3.2.4/api/python/reference/api/pyspark.sql.DataFrame.groupBy
df_group = unique_df.___________('age').___________(sf.count('*').alias('count')).sort('age')
df_group.show()  # 나이별로 그룹화하고 카운트 출력


In [ ]:
# SQL 쿼리 실행
df.___________("people")  # people이라는 이름의 임시 뷰 생성
# createOrReplaceTempView()는 DataFrame을 SQL 테이블처럼 사용할 수 있는 임시 뷰로 만듭니다.
# 참조: https://downloads.apache.org/spark/docs/3.2.4/api/python/reference/api/pyspark.sql.DataFrame.createOrReplaceTempView.html

spark.___________("SELECT name, count(*) FROM people WHERE name>'J' GROUP BY name").show()  # SQL 쿼리 실행
# sql() 메서드는 SQL 쿼리를 실행하고 결과를 DataFrame으로 반환합니다.
# 참조: https://downloads.apache.org/spark/docs/3.2.4/api/python/reference/api/pyspark.sql.SparkSession.sql.html


In [ ]:
# 열 생성 및 열 이름 변경
df_sub = df.___________("substring", df["name"].substr(0, 1))  # name 열의 첫 글자를 substring 열로 추가
# withColumn()은 DataFrame에 새로운 열을 추가하거나 기존 열을 수정하는 메서드입니다.
# substr()는 문자열의 서브스트링을 반환합니다.
df_sub.show()


In [ ]:
# withColumnRenamed()는 기존 열의 이름을 변경합니다.
# 참조: https://downloads.apache.org/spark/docs/3.2.4/api/python/reference/api/pyspark.sql.DataFrame.withColumnRenamed.html
df_initial = df_sub.___________("substring", "initial")  # substring 열을 initial로 이름 변경
df_initial.show()


In [ ]:
# 열 삭제
df_drop = df_initial.___________("age")  # age 열 제거
# drop()은 지정된 열을 DataFrame에서 제거합니다.
# 참조: https://downloads.apache.org/spark/docs/3.2.4/api/python/reference/pyspark.pandas/api/pyspark.pandas.DataFrame.drop.html

df_drop.show()

In [ ]:
# RDD 변환 및 Pandas 변환
rdd_out = df_initial.rdd  # DataFrame을 RDD로 변환
# rdd 속성은 DataFrame을 RDD 형식으로 변환합니다.
print(rdd_out.collect()[:10])  # RDD 출력

pandas_out = df_initial.toPandas() # RDD의 상위 10개 데이터 출력
# collect()는 RDD 또는 DataFrame의 모든 요소를 Python 목록으로 반환합니다.
print(pandas_out)


In [ ]:
spark.stop()  # Spark 세션 중지